<img src="./h2j_logo.png" style="height:500px"> 


# ============ Scroll to the bottom to see the "H2J" Demo =============

## Transpiler Documentation



### ================================ Background =============================================

- The "H2J" Transpiler is an open source transpiler capable of translating a subset of Haskell into Julia. I started this project on 7/5/2020 and was determined to have a workable transpiler by JuliaCon 2020, along with documentation. 

- No Haskell to Julia transpiler exists on GitHub. Additionally, much of the Julia ecosystem is concerned with 
   1. Machine learning + Optimization (which happens to be my background) most notably through Flux.jl, Optim.jl etc.
   2. Differentiable programming - ForwardDiff.Jl etc. 
   3. ODEs, PDEs - DifferentialEquations.jl etc. 
   
   and other related (and complementary) numerical/scientific computation based projects. 
   While these topics are what lead me to discover Julia, I also noticed Julia has strong metaprogramming capabilities, 
   and I could not find any transpilers which were written exclusively in Julia. 
   
   This notebook will allow user to read in a Haskell script as a string from a .txt file (or manually passing in the string)
   and transpile it into a working (or mostly working) Julia program. 

### ==========================================================================================
#### Haskell scripts are assumed to be syntatically correct as determined by the GHC. 

#### Haskell programs are fed into H2J as a string. 

#### H2J Is primilarily intended as a base/preliminary sketch of a full Haskell to Julia transpiler, but has enough 
#### functionality even in the first release to translate Haskell modules that include long lists of simple functions. 

####  If H2J is sufficiently popular, the repository will be frequently updated to include more functionality on a bimonthly basis. 

#### The rest of the cell will document the functionalities of H2J v1.0


### ========================== H2J TRANSPILER FEATURES ====================================

#### 1. Functions 
         - H2J is capable of translating single line Haskell functions written with a default number of arguments
           into Julia. While not yet tested, H2J may not work if the function's definition extends to more than one line. 
         
         - For now, only "standard" arithmetic operations are able to be translated.
           this includes addition, subtraction, multiplication, division, square roots and exponents.  
           That said, any single line statement in Haskell that is exactly identical in syntax to the Julia equivalent
           can be translated, but there may be an issue with the argument types. 
           
         - H2J is also capable of translating Haskell functions which include a "where" statement.
           In these cases, the Haskell function has two lines for the definition. 
           Ex. heron :: Float -> Float -> Float
               heron a b c = sqrt(s*(s-a)*(s-b)*(s-c))
                             where s = (a+b+c)/2
         
         - H2J retains the function name and the variables used, along with the appropriate types 
           when being translated into Julia. 
         
         - Note that in Haskell, ALL functions are treated as curried, which means each function only 
           takes one argument at most. Therefore, it is sufficient for the transpiler
           to take Haskell functions of the form f :: (a,b,...) -> z and 
           translate them first to a function of the form f :: a -> b -> ... -> z.  
          
           H2J v1.0 supports currying of functions of the form g :: (a,b) -> c to f = curry g 
           which is equivalent to f :: a -> b -> c 
           
           Additionally, H2J v1.0 supports partial applications of curried functions of the above form. 
            Ex. add :: Integer -> Integer -> Integer
                add x y = x+y
                addThree = add 3
            H2J can recognize partial applications of Haskell functions when there are exactly two arguments between
            the function definition and the "=" sign. Functions like "add x y" support partial applications but functions
            like "heron a b c" do not because it is a function of three arguments. 
         
          - Future releases of H2J will support all higher-order functions, be able to curry/uncurry an arbitrary
            number of functions, and support partial applications with the appropriate number of arguments. 
        
          
#### 2. Custom types

        - H2J does provide the ability to translate custom data types from Haskell into Julia, but with limitations.
           
           Ex. data Dice = One | Two | Three | Four | Five | Six - Custom Haskell data type
           Note that this data type can take on six values, delimited by the "|"s. 
           
           In Julia, this can be converted to a mutable struct (values are allowed to be changed) - 
           
           mutable struct Dice
              One
              Two
              Three
              Four
              Five
              Six
           end 
           
           One limitation is H2J does not support the "deriving" clause from Haskell, which is critical to describing
           behavior of "Dice". For instance, in Haskell, Six > Five can be made to be a true statement but this is 
           not directly apparent in the mutable struct. 
           
           Right now, the user would manually add functionality to the mutable struct to achieve the 
           same behavior. Additionally, mutable structs may not even be the most appropriate way 
           of handling custom classes, but for now, they are a workable translation. 
           
    =========== MAPPINGS OF TYPES in V1.0 ==========
              Haskell   ||   Julia
           -------------||-------------
            Integer     ||    Int64
            Double      ||    Float64
            Float       ||    Float64
            Maybe       ||    Any      
            
        The "Maybe" type in Haskell I have not yet decided how to handle, but for now, translating it to Julia's "Any" type
        seemed like a workable solution. 
          
#### 3. Map statements
         - Haskell is known for being able to express the same semantics through multiple syntactic options
           - ex."[ addFive x | x <- [1..5]]" is identical to "map addFive [1..5]". 
          H2J treats both statements as equivalent and uses the same function to convert them to the equivalent
          Julia map statement. 
         
         - So far, H2J is only tested for map statements of the following form: 
            - [ <function> <var> | <var> <- [a..b]]  
            - map <function> [a..b] where a, b are integers with  b >= a+2
            - A feature to soon add would be applying this syntax directly to user defined lists/arrays like [1,5,31]
            - Additionally, applying map statements to non-integer types will be supported in future releases. 
         
#### 4. Filter statements
        - Haskell filter statements can also easily be converted to Julia filter statements. 
        - H2J supports statements of the form: filter <condition> [a..b] 
          the conditions supported are comparison operators >,<,>=,<=, == and can be written with or without parantheses
        - H2J filter statements also translate filter even [a..b] and filter odd [a..b], again with or without parantheses
        - like with map statements, H2J will support filter statements 
          involving conditions and arguments of non-integer type in future releases. 
        
#### 5. Print statements 
        - H2J supports user defined print statements of the form "<statement that includes quotes>" 
        - H2J also supports translating the printing of function return values. 

#### 6. Code comments
        - H2J currently only ignores comments. The functions that parse the input were not written with the idea 
          to be able to handle comments on the same line. Translating Haskell comments into Julia comments will exist
          in future releases. 
          

         
          
### ========================== H2J CODE DOCUMENTATION ===============================


#### H2J V1.0 was a balancing act between getting working transpiler functionality and having the transpiler code be robust enough so that adding new features in the future would be easier. The main transpiler function includes many smaller helper functions you can find in the cells below this one and before the main H2J block. 

##### The main transpiler function itself will likely need to be refactored due to some inconsistencies in the design. 

##### These inconsistencies stem from using different logic to parse different Haskell blocks, rather than an overarching framework that can be applied to all Haskell blocks. For documentation purposes, the transpiler code is divided into sections, delimited by comments, which tell users what the goal of each block is, along with additional comments on important lines, describing the process the transpiler is achieving on those lines. This will make it easier for anyone who wishes to contribute by refactoring the H2J code. Finally, when the transpiler encounters a situation where errors have been found in the past, it will output a comment (in Julia) to alert the user of what the bug is (very minors found so far only). 


### ===========================   KNOWN ISSUES  ===============================================

###### Note this list used to much longer!

1) Printing the results of uncurried functions like f :: (Float,Float) -> Float in the "main" block of the Haskell script leads to an extra comma "," in the print statement

2) The h2j_transpile function works by reading a Haskell script that has been converted into a text file, and then outputs the translated julia code into a similarly named text file. Note that if putStr statements were used in the original Haskell script, there will be errors in the print statements (this is why the file converts to a .txt file and not a .jl file)

3) About eight other bugs have been fixed, but the time table I've worked on has been too short to say the compiler is flawless 
   besides the above two issues. In general, Haskell programs that stick to a similar style as seen in the demos is less likely to cause issues.  
  
  - Again, H2J will output helpful comments when one of these two situations may have been encountered to help the user 
    quickly debug the translated Julia code. The errors do not take long to fix should they arise. 
   




####  --------------------------------------------------- H2J v1.0 Design Philosophy ------------------------------------------------------------------------------

         - Before reading the rest of the section, it is important to discuss why the transpiler isn't written
           as a Julia function that inputs an Abstract Syntax Tree (AST) from Haskell and converts 
           to appropriate Julia code.
           
        =====================================================================================================
           1. The first reason is that a generated Haskell AST may not be uniform across compilers, 
               despite the high level Haskell code being identical. In practice, this is not a significant concern
               and so isn't as important as reasons 2 and 3.
           
           2. The second key reason is that it is not always apparent the best way to actually translate
              the Haskell AST into working Julia code. Even if we could translate it, that wouldn't 
              necessarily mean the resulting Julia code will be readable. Having readability
              of the resulting code should be a high priority, and as H2J shows, readable code 
              is possible without having to bother with ASTs. 
           
           3. The third reason is that for a transpiler to work well, it doesn't necessarily
           need to be able to translate the ENTIRE language, only "statements that are syntatically isomorphic". 
           Let's break that down. 
           
           Suppose S is the set of all possible Haskell statements, and S\~ is the set of all Haskell statements that 
           are not equivalent to any other Haskell statement. A transpiler for S\~ would require less code (fewer situations
           that are relevant) and would work just as well. Haskell, being an extremely versatile language, allows users
           write code with the same semantic meaning in a uniquely wide variety of syntactic styles, which is not found
           to the same degree in many other languages. 
           
           Ex. [ f | x <- [1,2,3]] is semantically equivalent to map f [1,2,3] although the syntax clearly differs. 
           
           Taking this into account, the architecture design for H2J could be such that it takes in Haskell programs of a
           particular syntactic style and converts them to clean, readable Julia code.
           
          =============================  COMPILER ARCHITECTURE / DESIGN =========================================
          
          Rather than using one type of parser for the entire transpiler, different approaches were used for 
          different types of Haskell statements. The compiler itself acts as a sequence of lexer-parser pairs which
          are context-sensitive to translate Haskell code. In the initial design, consideration was given whether to use
          an LL parser or an LR parser but eventually it was decided to use different parsers for different subsets
          of the language and chain them together (although in an ad-hoc, not necessarily logical fashion). 
          
          In future iteration of H2J, there will be more careful consideration into available parsing schemes. 
          Given how H2J v1.0 was only meant to translate a subset of Haskell into Julia, this was acceptable to me. 
          
          ======================================================================================================
         - Most of the functions work by making heavy use of Julia filter, strip, and split statements. 
         - The split statement is vital to split the Haskell code into lexemes/tokens. 
         - The strip statement is applied as needed to remove white spaces so that functions work with the
           corresponding tokens, regardless of the white spaces between the tokens in the original code. 
           
         - The filter statement is used extensively in the main H2J compiler. 
         - The filter statements help identify which Haskell lines, or several line contain curry statements,
           partial applications, map statements etc. 
           
         - This is crucial, because we don't know beforehand how many such statements exist in the original
           Haskell program. Do we have one map statement? Twenty? We need a way to filter out an arbitrary number
           of statements. 
          
         - Loops are used in conjunction with filter statements because we may need to apply a helper function N number
           of times and we do not know beforehand how many. 
          
         - To translate Haskell functions into Julia, H2J works by taking the Haskell function declaration and defintion
           as two seperate statements which are then converted to the corresponding Julia function. The reasoning behind
           this approach is that Julia is dynamically typed, wheras Haskell is statically typed. The arguments passed to
           a Haskelll functions must have the appropriate type, whereas in Julia, many types can be supported for the same
           argument depending on the function.
           
           Ex. add(x,y) supports both Int64, and Float64 in Julia, but in Haskell it can only support one or the other.
           Julia fortunately allows type assertion statements, so that the function checks to make sure the arguments
           are indeed the correct types to be passed. 
           If we want integer only addition: 
           
           Haskell declaration - add:: Integer -> Integer -> Integer 
           Julia declaration - function add(x::Int64,y::Int64) 
           
           If we wanted addition for floats:
           
           Haskell declaration - add:: Float -> Float -> Float 
           Julia declaration - function add(x::Float64,y::Float64)
           
           Other functions work similarly...
          
           
           



#### There are cells below which contain some early H2J code where I was experimenting translating simple statements from Haskell to Julia and vice-versa. There could be some old functions which may prove useful for future releases, but there is a lot of "scratch work" present in the cells. 

In [1]:
macro hello_world_haskell()
    return println("main = do\n", " putStrLn"," \"","Hello","\"\n putStrLn ", "\"", "World!", "\"")
end
    

function parse_hs(func::String)
    #parsing a Haskell function
    code = fun
    check = occursin("::",code)
    if check == false 
        println("incorrect syntax or format unsupported")
    end
    if check == true
        println("Haskell function declared successfully...")
        block = split(code,"::")
        block2 = split(block,)
    end
    return block
end

macro haskell_list(block::String)
    code = split(block,"[")
    code2 = split(code[2],"...")
    start = parse(Int32,code2[1][1])
    terminal_str = split(code2[2],']')
    terminal = parse(Int32,terminal_str[1])
    return collect(start:terminal)
end


macro hs_head(block::String)
    return string(block[2])
end

macro cons(block::String)
    #Haskell-to-Julia
    code = split(block,":")
    start = parse(Int32,code[1])
    arr_init = parse(Int32,code[2][2])
    arr_term = reverse(code[2])[2]
    t = arr_init, parse(Int32,arr_term)
    arr = vcat(start,LinearIndices((t[1]:t[2])))
    return arr
end

macro main_do()
    return println("main = do")
end

macro hs_print(str::String)
    return println(" putStrLn"*" \""*str*"\"")
end

macro hs_read_in(prompt::String)
    println(prompt)
    r = chomp(readline())
    #add type conversion as needed
    return r
end

macro hs_io(prompt::String,var_name)
    var_name = string(var_name)
    head = "putStrLn"*" \""*prompt*"\""
    println(head)
    println(" ",var_name*" "*"<- getLine")
    return var_name
end

macro hs_factorial_def()

    return println(" fac n = if n == 0 then 1 else n * fac (n-1)")
end

macro new_type(name::String)
    return println(" data"*" "*name*" =")
end

macro new_type_vals(name1::String,name2::String)
    return println(" "*name1* " | "*name2)
end
p= @haskell_list("[1,2...300]")

r= @hs_head("[1,2,3]")

l = @cons("0:[1,2,3,4,5]")

test = @hello_world_haskell



main = do
 putStrLn "Hello"
 putStrLn "World!"


In [2]:
macro get_args(funcall::Expr)
    funcall.head == :call || error("not a function call")
    return funcall.args[2:end]
end

macro getargs(f)
    args=Base.method_argnames(first(methods(f)))
    return args
end

function translate_types(var)
    if typeof(var) == Int
        return "Integer"
    end
    if typeof(var) == float
        return "Float"
    end
end

mutable struct construct_type
    #new data types can be made with structs and translated to Haskell easily
    name::String
    T1::String
    T2::String
    new_type(name,T1,T2) = new(name,T1,T2)
end

function struct_to_typeclass(S)
    #pass in a struct
    T = new_type.name
    val1 = new_type.T1
    val2 = new_type.T2
    println("data ", T, " = ", val1, " | ", val2 )
end

    
translate_types(4)


macro jl2hs_type(T)
    #get methods/attributes from T, parse and translate into Haskell
    return 0
end


@jl2hs_type (macro with 1 method)

### Hello world program below from macros

In [3]:
@main_do
@hs_print("Hello")
@hs_print("World!")


main = do
 putStrLn "Hello"
 putStrLn "World!"


In [4]:
args = Base.method_argnames(first(methods(blah)))

UndefVarError: UndefVarError: blah not defined

In [5]:
function add(x,y)
    result = x+y::Int64
    T = typeof(result)
    return result, T
end
function blah(x,y,z)
    return x,y,z
end

 #works to get arguments

function jl2hs_fun(jl_func)
    #Note that Julia is dynamically type and so the types are inferred at run-time NOT compile time
    #Haskell is statically typed so we need to manually specify the arguments 
    args = Base.method_argnames(first(methods(jl_func))) #returns args as an array of symbols
    name = string(jl_func)
    return args
end
function hs2jl_type(t::String)
    if t == filter(t -> !isspace(t), "Integer")
        return Int32
    end
    if t == filter(t -> !isspace(t), "float")
        return real
    end
    if t == filter(t -> !isspace(t), "Maybe" )
        return Any
    end
    return t
end

function map_types(type_array::Array)
    return map(hs2jl_type,type_array)
end
        
function hs2jl_fun(declaration::String,definition::String)
    dec = split(declaration,"::")
    name = dec[1]
    argtypes = count("->",dec[2])
    jl_types = []
    for i = 1:argtypes
        arg = split(dec[2],"->")[i]
        arg = lstrip(arg)
        if arg != "Integer" #this doesn't work as intended
            println("test")
            println(arg)
            append!(jl_types,[Int32])
        end
    end
    jl_dec= string("function ", name,"", "(", "x::",jl_types[1],",y::",jl_types[2],")")
    def = split(definition,"=") #parse definition on = sign
    jl_def = string(" return ", def[2])
    jl_fun = string(jl_dec,jl_def)
    println("return type",dec)
    return jl_fun
end
    

    
#translation = jl2hs_fun(add)
test=hs2jl_fun("add:: Integer -> Integer -> Maybe", "add x y = x+y")

function jl2hs_fun(func::String) #works for functions with two arguments
    parse = split(func,")")
    dec = parse[1]
    def = parse[2]
    n= lstrip(split(dec,"function")[2])
    name = split(n,"(")[1]
    two_args = split(n,",")
    second_argparse = split(two_args[2],"::") #gives variable name and type of variable for second argument 
    first_argparse = split(two_args[1],"::")
    first_arg = split(first_argparse[1],"(")[2]
    first_type = first_argparse[2]
    second_arg = second_argparse[1]
    second_type = second_argparse[2]
    if first_type == "Int32"
        hs_first_type = "Integer"
    end
    if first_type == "real"
        hs_first_type == "Float"
    end
    if first_type == "any"
        hs_first_type = "Maybe"
    end
    if second_type == "Int32"
        hs_second_type = "Integer"
    end
    if second_type == "real"
        hs_second_type == "Float"
    end
    if second_type == "any"
        hs_second_type = "Maybe"
    end
    
    hs_fun = string(name," :: ",hs_first_type," -> ", hs_second_type," -> ", "Integer")
    return hs_fun
end

test2 = jl2hs_fun("function add(x::Int32,y::Int32)"*"return x+y")
#array = ["Integer","Integer","Integer"]
#map_types(array)

test
Integer 
test
Integer 
return typeSubString{String}["add", " Integer -> Integer -> Maybe"]


"add :: Integer -> Integer -> Integer"

In [6]:
function subtract(x::Int32,y::Int32)
    return x-y
end

#haskell_subtract = jl2hs_fun("function multiply(x::Int32,y::Int32)"*"return x*y")

julia_subtract = hs2jl_fun("subtract::Integer -> Integer -> Integer","subtract x y = x-y")



test
Integer 
test
Integer 
return typeSubString{String}["subtract", "Integer -> Integer -> Integer"]


"function subtract(x::Int32,y::Int32) return  x-y"

In [7]:
function jl2hs_dec(decl::String)
    dec = split(decl,"(")
    args = split(dec[2],",")
    first = split(args[1],"::")
    second = split(args[2],"::")
    first_arg = first[1]
    first_argtype = first[2]
    second_arg = second[1]
    second_argtype = split(second[2],")")[1]
    out_parse = split(args[2],")")
    out_type = split(out_parse[2],"::")[2]
    name = lstrip(chop(dec[1],head=8,tail=0))
    println(first_argtype)
    if first_argtype == "Int32" # || or operations
        T1 = "Integer"
        
    elseif first_argtype == "real" || first_argtype == "Float32" || first_argtype == "Float64"
        T1 = "Float"
    
    elseif first_argtype == "Any"
        T1 = "Maybe"
    end
    if second_argtype == "Int32"
        T2 = "Integer"
    
    elseif second_argtype == "real" || first_argtype == "Float32" || first_argtype == "Float64"
        T2 = "Float"
    
    elseif second_argtype == "Any"
        T2 = "Maybe"
    end
    if out_type == "Int32" || out_type == "Int64"
        T_out = "Integer"
    
    elseif out_type == "real" || first_argtype == "Float32" || first_argtype == "Float64"
        T_out = "Float"
    elseif out_type == "Any"
        T_out = "Maybe"
    end
    return string(name," :: ", T1 ," -> ", T2, " -> ", T_out)
end

test = jl2hs_dec("function subtract(x::Float32,y:Int32)::Int64")

Float32


"subtract :: Float -> Float -> Integer"

In [8]:
function hs2jl_dec(decl::String)
    dec = split(decl,"::")
    name = rstrip(dec[1])
    args = split(dec[2],"->")
    jl_args = []
    for i = 1:length(args)
        arg = lstrip(rstrip(args[i]))
        if arg == "Float"
            T1 = "Float64"
        elseif arg == "Integer"
            T1 = "Int64"
        elseif arg == "Maybe"
            T1 = "Any"
        end
        append!(jl_args,[T1])
    end
    jl_dec = string("function ",name,"(x::",jl_args[1],",","y::",jl_args[2],")","::",jl_args[3])
    return println(jl_dec)
end

test2=hs2jl_dec("add :: Integer -> Integer -> Integer")

function add(x::Int64,y::Int64)::Int64


In [9]:
function hs2jl_def(def::String)
    parsed = split(def,"=")
    return println(" return ",lstrip(parsed[2]))
    
end
hs2jl_def("add x y = x+y")

function hs2jlfun(dec::String,def::String)
    jl_dec = hs2jl_dec(dec)
    jl_def = hs2jl_def(def)
    println("end")
end
function jl2hsfun(dec::String,def::String)
    parsed_dec = split(dec,"(")
    name = lstrip(chop(parsed_dec[1],head=8,tail=0))
    args = 0
    
end
jl2hsfun("function add(x::Int32,x::Int32)::Int32","return x+y")

 return x+y


0

### Haskell to Julia function definition and declaration w/ two arguments

#### The code below takes a Haskell function of two arguments and outputs the corresponding Julia function.
#### Right now, only basic arithmetic functions are supported, but with time we will improve upon this 

In [10]:
hask_dec = "areaRect :: Float -> Float -> Float"
hask_def = "areaRect x y = x*y"

hs2jlfun(hask_dec,hask_def)



function areaRect(x::Float64,y::Float64)::Float64
 return x*y
end


In [11]:

macro hs_fib()
    return println("fib 0 = 0 
        fib 1 = 1
fib n = fib (n-1) + fib (n-2)")
end


macro pm_fact()
    return println(" factorial 0 = 1 
 factorial n = n * factorial (n - 1)")
end



@pm_fact (macro with 1 method)

#### The cells from here on out are essential to H2J v1.0

In [12]:
hs_code = "heron :: Float -> Float -> Float -> Float
           heron a b c = sqrt(s*(s-a)*(s-b)*(s-c))
                     where s = (a+b+c)/2"

function where_functions(hs_fun::String)
    lex = split(hs_fun,"::")
    name = strip(lex[1])
    lex2 = split(lex[2],"\n")
    args = split(lex2[1],"->")
    args = map(x -> strip(x),args)
    jl_types = []
    for i = 1:length(args)
        if args[i] == "Double"
            append!(jl_types,["Float64"])
        elseif args[i] == "Float"
            append!(jl_types,["Float64"])
        elseif args[i] == "Integer"
            append!(jl_types,["Int64"])
        elseif args[i] == "Maybe"
            append!(jl_types,["Any"])
        elseif args[i] != "Integer" || "Double" || "Float" || "Maybe"
            append!(jl_types,args[i])
        end
    end
    if args[end] == "Double"
            return_type = "Float64"
        elseif args[end] == "Float"
            return_type= "Float32"
        elseif args[end] == "Integer"
            return_type = "Int64" 
        elseif args[end] == "Maybe"
            return_type = "Any" 
        elseif args[end] != "Integer" || "Double" || "Float" || "Maybe" 
            return_type = args[end]
        end
    
    names_and_def = strip(lex2[2])
    names_and_def = split(names_and_def,"=")
    def = strip(names_and_def[2])
    names = names_and_def[1]
    args = split(names,name)
    arg_names = strip(args[2])
    arg_names = filter(x -> !isspace(x), arg_names)
    fun_body = []
    for i = 1:length(arg_names)
        append!(fun_body,[string(arg_names[i],"::",jl_types[i],",")])
    end
    fun_body[end]=chop(fun_body[end],head=0,tail=0)
    ret =split(lex[2],"where")
    wh = strip(ret[2])
    ret = strip(split(ret[1],"=")[2])
    jl_function_dec = string(chop(string("function ",strip(name),"(",join(fun_body,""),")"),head=0,tail=2),")::",return_type)
    println(jl_function_dec)
    println("   ",wh)
    println("   return ",ret)
    println("end")
end
where_functions(hs_code)

function heron(a::Float64,b::Float64,c::Float64)::Float32
   s = (a+b+c)/2
   return sqrt(s*(s-a)*(s-b)*(s-c))
end


In [13]:
function make_julia_ast(jl_code::String)
    expr = dump(jl_code)
    return expr
end


make_julia_ast (generic function with 1 method)

In [14]:
function lexer(code::String)
    main_parse = split(code,"main :: IO ()") #splits code before main and after main :: IO ()
    pre_main, main = main_parse[1], main_parse[2]
    return main_parse
end

lexer(haskell_code)


UndefVarError: UndefVarError: haskell_code not defined

In [15]:
p = "arr = [addOne x | x <- [1..12]]"

code = "arr = map addOne [1..12]"
function hsmap_convert(hs_code::String)
    #function to convert Haskell's map syntax to the list comprehension equivalent
    var, hs_code = split(hs_code,"=")
    sp = split(hs_code,"map")
    func_name = strip(split(sp[2],"[")[1])
    arr = strip(split(sp[2],func_name)[2])
    conv = string(var," = ","[",func_name, " x | x <- ",arr,"]")
    hs_map2list(conv)
end
    
    
hsmap_convert(code)

UndefVarError: UndefVarError: hs_map2list not defined

In [16]:
hs_code = "arr = [addFour a | a <- [1..12]]"
function addFour(x::Int64)
    return x+4
end
function hs_map2list(hs_code::String)
    code = split(hs_code,"=")
    name, lst = strip(code[1]), strip(code[2])
    func_name, m = split(lst,"|")[1], split(lst,"|")[2]
    func_name = strip(split(func_name,"[")[2])
    m, lst = split(m,"<-")[1], split(m,"<-")[2]
    m, lst = strip(m), strip(lst)
    arr = chop(lst,head=1,tail=2)
    arr = split(arr,"..") #try / catch block for user-defined lists like [1,2,3,4]
    a, b = map(x->parse(Int64,x),arr)
    func_name = filter(x -> !isspace(x),func_name)
    user_def = filter(x -> !occursin(x,"x"),func_name)
    println("")
    func_name = chop(func_name,head=0,tail=1)
    out = string(name, " = map", "(", m," -> ",func_name,"(",m,"), ", "collect(",a,":",b,"))")
    
    
end

hs_map2list(hs_code)

"arr = map(a -> addFour(a), collect(1:12))"

In [17]:
many_args= "quad_perimter :: Float -> Float -> Float -> Float -> Float
            quad_perimter a b c d = a+b+c+d"

"quad_perimter :: Float -> Float -> Float -> Float -> Float\n            quad_perimter a b c d = a+b+c+d"

In [18]:
function haskell_to_julia_function(hs_fun::String) #parses individual functions for any number of arguments
    name, dec = split(hs_fun, "::")
    def = split(hs_fun,"\n")
    def = strip(def[2])   
    args = split(def,"=")
    args= chop(args[1],head=length(name),tail=0)
    args = filter(x -> !isspace(x), args) #removes spaces from arguments 
    body = dec
    arg_types = split(body,"->") #last index contains the function definition and the first n-1 terms are types
    def = split(arg_types[end],"\n")
    return_type = strip(def[1])
    if return_type == "Float"
        return_type = "Float64"
    elseif return_type == "Double"
        return_type = "Float64"
    elseif return_type == "Integer"
        return_type = "Int64"
    elseif return_type == "Maybe"
        return_type = "Any"
    end
    jl_types = []
    for i = 1:length(arg_types)-1
        T = lstrip(rstrip(arg_types[i]))
        if T == "Float"
            append!(jl_types,["Float64"])
        elseif T == "Double"
            append!(jl_types,["Float64"])
        elseif T == "Integer"
            append!(jl_types,["Int64"])
        elseif T == "Maybe"
            append!(jl_types,["Any"])
        elseif T != "Float" || "Integer" || "Maybe"
            append!(jl_types,[T])
        end
    end
    jl_types = jl_types[:]
    jl_args = []
    for j = 1:length(jl_types)
        append!(jl_args,[string(args[j],"::",jl_types[j],",")])
    end
    function_def = [string(rstrip(string("function ",name)),"(")]
    jl_args = permutedims(jl_args)
    for k = 1:length(jl_types)
        append!(function_def,[string(jl_args[k])])
    end
    #code to turn return type into Julia type
    last_arg = chop(function_def[end],head=0,tail=1)
    function_def[end] = string(last_arg,")","::",return_type)
    jl_function_dec = string(chop(string("function ",strip(name),"(",join(jl_args,""),")"),head=0,tail=2),")::",return_type)
    def = split(body,"=")[2]
    println(jl_function_dec)
    println("    return ",def)
    println("end")
end

    
r=haskell_to_julia_function(many_args)



function quad_perimter(a::Float64,b::Float64,c::Float64,d::Float64)::Float64
    return  a+b+c+d
end


In [19]:
filt_1 = "arr = filter (even) [1..500]"
filt_2 = "arr = filter (==1) [1..20]"

function hs_filter(hs_code::String)
    name, fil = split(hs_code,"=")
    name = strip(name)
    code = split(hs_code,"filter")
    cond = strip(code[2])
    arr = split(cond,"[")
    arr_2 = split(arr[2],"]")
    nums = split(arr[2],"..")
    b = strip(split(nums[2],"]")[1])
    a = strip(nums[1])
    ran = map(x->parse(Int64,x),[a,b])
    cond = strip(split(cond,"[")[1])
    
    if length(filter(x -> occursin(x,"even"),cond)) != 0
        condition = "%2 == 0"
        jl_statement = string(name, " = ","filter(x -> x",condition,", collect(",ran[1],":",ran[2],"))")
    elseif length(filter(x -> occursin(x,"odd"),cond)) != 0
        condition = "%2 == 1"
        jl_statement = string(name, " = ","filter(x -> x ",condition,", collect(",ran[1],":",ran[2],"))")
    elseif !occursin(cond,"even") || !occursin(cond,"odd") == false
        cond = filter(x -> !occursin(x,"("),cond)
        cond = filter(x -> !occursin(x,")"),cond)
        jl_statement = string(name," = ", "filter(x -> x",cond,", collect(",ran[1],":",ran[2],"))")
    end
end

hs_filter(filt_1)

"arr = filter(x -> x%2 == 0, collect(1:500))"

In [20]:
suit = "data Suit = Club | Diamond | Heart | Spade"
card_val = "data cardValue = Two | Three | Four | Five | Six | Seven | Eight | Nine | Ten | Jack | Queen | King | Ace"
data_card = "data Card = Card{value:: cardValue , suit:: Suit}"

#omitting the deriving portion for now

function haskell_custom_dtype(hs_type::String)
    if occursin("{",hs_type) == false
        data = split(hs_type,"=")
        name = lstrip(chop(data[1],head=4,tail=1))
        types = split(data[2],"|")
        declr = string("mutable struct ",name)
        println(declr)
        for i = 1:length(types)
            println(types[i])
        end
        println("end")
        data = split(hs_type,"=")
    elseif occursin("{",hs_type) == true
        data = split(hs_type,"=")
        name = lstrip(chop(data[1],head=4,tail=1))
        def = lstrip(data[2])
        
    end
end

#julia_suit = haskell_custom_dtype(suit)
#julia_card_val = haskell_custom_dtype(card_val)
julia_card = haskell_custom_dtype(data_card)

"Card{value:: cardValue , suit:: Suit}"

In [21]:
st="print(heron 1.2 2.5 5.6)"

st2 = "putStr \"Sum of x,y = \""
function hs_print(hs_code::String)
    body = split(hs_code,"print")[2]
    body = filter(x -> !occursin(x,"("),body)
    body = filter(x -> !occursin(x,")"),body)
    body = split(body," ")
    func_name = body[1]
    num_args = length(body)-1
    args = []
    for i = 1:num_args
        append!(args,[body[i+1]]) 
    end
    arg_statement = join(args,",")
    jl_statement = string("println(",func_name,"(",arg_statement,"))")
    println(jl_statement)
    return jl_statement
end

hs_print(st)

function hs_string_print(hs_code::String)
    statement = split(hs_code,"putStr")[2]
    statement = strip(statement)
    statement = filter(x -> !occursin(x,""""""),statement)
    jl_statement = string("println(",statement,")")
    println(jl_statement)
    return jl_statement
end
hs_string_print(st2)

println(heron(1.2,2.5,5.6))
println("Sum of x,y = ")


"println(\"Sum of x,y = \")"

### Conditionals did NOT make into H2J v1.0 but will be in the next update

In [22]:
code = "doubleSmallNumber x = (if x > 100 then x else x*2)"

code2 = "tripleBigNumber x = (if x < 100 then x else 3*x)"


function hs_conditionals(hs_code::String) #works for functions with if else in their definition 
    func, conds = split(hs_code,"=")
    conds = split(conds,"else")
    st_if = filter(x -> !occursin(x,"("),conds[1])
    st_if = strip(st_if)
    st_if = split(st_if,"then")
    
    st_if = string(st_if[1]," == true\n","      ",strip(func)[end]," =",st_if[2])
    func_name = strip(func)
    func_name = filter(x -> !isspace(x),func_name)
    func_name = chop(func_name,head=0,tail=1)
    st_else = strip(filter(x -> !occursin(x,")"),string(conds[2])))
    block_else = split(conds[1],"if")
    block_else = strip(split(block_else[2],"then")[1])
    st_else = string("else ",block_else, " == false\n","      ",strip(func)[end]," = ",st_else)
    func_dec = string("function ",func_name,"(",strip(func)[end],")")
    func = string(func_dec,"\n   ", st_if,"\n   ", st_else,"\n","   end\n","   return ",strip(func)[end],"\nend")
    println(func)
    
    
    
end
hs_conditionals(code2)
    

function tripleBigNumber(x)
   if x < 100  == true
      x = x
   else x < 100 == false
      x = 3*x
   end
   return x
end


In [23]:
function tripleBigNumber(x)
   if x < 100  == true
      x = x
   else x < 100 == false
      x = 3*x
   end
   return x
end

tripleBigNumber(450)

1350

### Currying functions

In [24]:
code = "f :: (Integer, Integer) -> Integer\ng (a,b) = a^2 + b^2\nf = curry g"
println(code)

function hs_curry(hs_code::String) #function for currying
    func_dec, func_def, curr = split(hs_code,"\n")
    ret_type = strip(split(func_dec,"->")[2])
    if ret_type == "Integer" 
        ret_type = "Int64"
    elseif ret_type == "Float"
        ret_type = "Float64"
    elseif ret_type == "Float"
        ret_type = "Float64"
    elseif ret_type == "Maybe"
        ret_type = "Any"
    elseif ret_type != "Maybe" || "Integer" || "Float" || "Double" 
        ret_type = ret_type
    end
    name, args = split(func_dec,"::")
    args = strip(split(args,"->")[1])
    arg1, arg2 = split(args,",")
    arg1, arg2 = chop(arg1,head=1,tail=0), strip(chop(arg2,head=0,tail=1))
    if arg1 == "Integer"
        arg1_type = "Int64"
    elseif arg1 == "Float"
        arg1_type = "Float64"
    elseif arg1 == "Double"
        arg1_type = "Float64"
    elseif arg1 == "Maybe"
        arg1_type = "Any"
    elseif arg1 != "Maybe" || "Integer" || "Float" || "Double" 
        arg1_type = arg1
    end
    if arg2 == "Integer"
        arg2_type = "Int64"
    elseif arg2 == "Float" 
        arg2_type = "Float64"
    elseif arg2 == "Double"
        arg2_type = "Float64"
    elseif arg2 == "Maybe"
        arg2_type = "Any"
       
    elseif arg2 != "Maybe" || "Integer" || "Float" || "Double" 
        arg2_type = arg2
    end
    uncurried, def = split(func_def,"=")
    uncurr_name, args = split(uncurried,"(")
    curr_name = strip(split(curr,"=")[1])
    arg_name1, arg_name2 = split(args,",")
    arg_name1 = filter(x -> !occursin("(",x),[arg_name1])[1]
    arg_name2 = chop(strip(arg_name2),head=0,tail=1)
    jl_dec = string("function ",curr_name,"(",strip(uncurr_name),"(",arg_name1,"::",arg1_type,",",arg_name2,"::",arg2_type,"))::",ret_type)
    jl_def = strip(def)
    out = string(jl_dec,"\n   ","return ",jl_def,"\n","end")
    orig_dec = string("function ",strip(uncurr_name),"(",arg_name1,"::",arg1_type,",",arg_name2,"::",arg2_type,")")
    orig_def = string("  return ", jl_def)
    orig_fun = string(orig_dec,"\n  ",orig_def,"\nend")
   
    curried_fun_dec = string("function ",curr_name,"(","c::Tuple)::",ret_type)
    curried_fun_def = string("return ",strip(uncurr_name),"(c[1],c[2])")
    curried_function = string(curried_fun_dec,"\n    ",curried_fun_def,"\nend")
    println(orig_fun,"\n")
    println(curried_function)
end

hs_curry(code)

f :: (Integer, Integer) -> Integer
g (a,b) = a^2 + b^2
f = curry g
function g(a::Int64,b::Int64)
    return a^2 + b^2
end

function f(c::Tuple)::Int64
    return g(c[1],c[2])
end


In [25]:
function g(a::Int64,b::Int64)
    return a^2+b^2
end

function f(c::Tuple)::Int64
    c = g(c[1],c[2])
    return c
end  



f (generic function with 1 method)

#### Partial application 

In [26]:
par_app = "addOneTwenty = add 120"
function partial_app(hs_code::String)
    var, func = split(hs_code,"=")
    func = strip(func)
    func = split(func," ")
    name, arg = func[1], func[end]
    result = string(strip(var),"(x",")"," = map(x -> ",name,"(",arg,",x","))")
    
end
    
partial_app(par_app)

"addOneTwenty(x) = map(x -> add(120,x))"

In [27]:
addOneTwenty(x) = map(x -> add(x,120),x)

test = addOneTwenty(5) #result should be 125

(125, Int64)

In [28]:
fun1 = "f x = x+1" #note without the space the two are syntactically equivalent 
fun2 = "add = add 1"
#====
test1 = strip(split(fun1,"=")[1])
test2 = strip(split(fun2,"=")[1])

test1 = filter(x -> occursin(" ",x),[test1])
test2 = filter(x -> occursin(" ",x),[test2])

test1 = filter(x -> occursin(" ",x),[strip(split(fun1,"=")[1])]))
test2 = filter(x -> occursin(" ",x),[strip(split(fun2,"=")[1])])
=#
function partial_check(func::String)
    val = filter(x -> occursin(" ",x),[strip(split(func,"=")[1])])
    if length(val) == 0
        return true
    end
    if length(val) == 1
        return false
    end
end
partial_check(fun2)

true

# =====================  H2J Demo ======================= #

In [29]:
haskell_code="add :: Integer -> Integer -> Integer
add x y = x + y

sphere_volume :: Float -> Float -> Float
sphere_volume r h = (4/3)*(3.1415)*(r^3*h)

triangleArea :: Float -> Float -> Float
triangleArea b h = (1/2)*b*h

addfourInts :: Integer -> Integer -> Integer -> Integer -> Integer
addfourInts w x y z = w+x+y+z

heron :: Double -> Double -> Double -> Double
heron a b c = sqrt(s*(s-a)*(s-b)*(s-c))
              where s = (a+b+c)/2


testFun :: Integer -> Integer -> Integer
testFun a b = sqrt(a)+s
              where s = 2*b

-- this is a comment
--this is another comment 

addOne :: Integer -> Integer
addOne a = a +1 

sumSquares :: (Integer,Integer) -> Integer
sumSquares (x,y) = x^2+y^2

curriedSquares = curry SumSquares

arr = [addOne a | a <- [1..200]]

arr2 = [addOne x | x <- [1..8]]

arr3 = filter (even) [1..500]

arr4 = filter (==1) [1..40]

arr5 = map addOne [1..12]

arr6 = map addOne [1..20]

addThree = add 3

addEight = add 8
data Suit = Club | Diamond | Heart | Spade
data cardValue = Two | Three | Four | Five | Six | Seven | Eight | Nine | Ten | Jack | Queen | King | Ace

-- this is a comment 

--this is another comment

main :: IO () 
main =  do 
putStr \"Sum of x,y = \"
print(add 10 25)
putStr \"Heron result = \"
print(heron 2.5 2.2 1.1)"

function main_parse(code::String)
    pre_main, main = split(code,"main")
    pre_main_funcs = split(pre_main, "\n") #next function's name always occurs after the "::"
    return pre_main,main 
end

#================================================ H2J MAIN ================================================================
==========================================================================================================================#


function hs2jl_compiler(hs_code::String) #========MAIN H2J FUNCTION ===============================#
    
    hs_code = filter(x -> x != "--",hs_code) #ignores all comments on all lines
    pre_processed_code = split(hs_code,"\n") 
    #================== CURRIED FUNCTIONS ========================
    ==============================================================
    =#
    if occursin("::",string(pre_processed_code)) && occursin(",",string(pre_processed_code)) == true #search for uncurried
        uncurried_decs = filter(x -> occursin("::",string(x)),pre_processed_code)
        uncurried_decs = filter(x -> occursin(",",string(x)),uncurried_decs)
        uncurried_defs = filter(x -> occursin("(",string(x)),pre_processed_code)
        uncurried_defs = filter(x -> occursin(",",string(x)),uncurried_defs)
        uncurried_defs = filter(x -> !occursin("::",string(x)),uncurried_defs) #filters out the uncurried declarations
        curry_statements = filter(x -> occursin("curry",string(x)),pre_processed_code)
    end
    for i = 1:length(curry_statements)
        statement = string(uncurried_decs[i],"\n",uncurried_defs[i],"\n",curry_statements[i])
        jl_curried = hs_curry(statement)
       
    end
    println("\n")
    #================ SCOPE SPLIT (PRE-MAIN AND MAIN) ====================
    ======================================================================#
    pre_main, main = split(hs_code,"main ::") #splits the scope into pre-main and main
    
    #===PRE-MAIN W/O FUNCTIONS CONTAINING "WHERE" KEYWORD ================
    ======================================================================
    =#
    if occursin("where",pre_main) == false #if functions with "where" keyword aren't found, translate the one-line functions
        partials = []
        pre_main_funcs = split(pre_main,"\n")
        lines = filter(x -> x !="", pre_main_funcs) #filters out the white spaces line by line
        custom_types = filter(x -> occursin("|",x), lines) #gets custom data types
        custom_types = filter(x -> !occursin("[",x),custom_types)
        funcs = filter(x -> !occursin("|",x),lines) #functions 
        funcs = filter(x -> (!occursin("(",x) || !occursin(",",x)),funcs) #removes curried function blocks
        funcs = filter(x -> !occursin("filter",x),funcs) #removes filter statements  
        funcs = filter(x -> !occursin("map",x),funcs) #removes map statements 
        funcs = filter(x -> !occursin("]",x),funcs)
        funcs = filter(x -> !occursin("curry",x),funcs) #removes curry statements
        funcs = filter(x -> (!occursin("(",x) || !occursin(",",x)),funcs) #removes curried function blocks
        hs_decs = filter(x -> occursin("::",x),funcs) #gets function definitons
        hs_defs = filter(x -> occursin("=",x),funcs) #gets function declarations
        #==== FUNCTIONS & PARTIALS ====#
        for i = 1:length(hs_defs)
            if partial_check(string(hs_defs[i])) == true
                println(partial_app(string(hs_defs[i])))
                println("\n")
                append!(partials,[hs_defs[i]])
            end
        end
        if length(partials) != 0
            for j = 1:length(partials)
                hs_defs = filter(x -> x != partials[j],hs_defs) #filtering out partial application statements from hs_defs
            end
            
        end
        for j = 1:length(hs_decs)
            haskell_to_julia_function(string(strip(hs_decs[j]),"\n",strip(hs_defs[j]))) #will have same length always
            println("")
        end
        #===== CUSTOM DATA TYPES =====#
        for k = 1:length(custom_types) #now translate the custom Haskell data types as Julia mutable structs 
            haskell_custom_dtype(string(custom_types[k])) 
            println("")
        end
        #===== LIST/ARRAYS ======#
        if occursin("[",pre_main) == true && occursin("|",pre_main) && occursin("=",pre_main) == true #find list comp arrays
            arrs = filter(x -> occursin("[",x), lines)
            arrs = filter(x -> !occursin("filter",x),arrs) #remove filter statements 
            arrs = filter(x -> !occursin("map",x),arrs) #remove map statements 
            for k = 1:length(arrs)
                println(hs_map2list(string(arrs[k]))) 
                
            end
        end
        println("\n")
        #==== HASKELL "FILTER" STATEMENTS ======
        ========================================#
        filters = filter(x -> occursin("filter",x),lines) 
        for i = 1:length(filters)
            println(hs_filter(string(filters[i])))
            println("\n")
        end
        #===== HASKELL "MAP" STATEMENTS =======
        =======================================#
        maps = filter(x -> occursin("map",x),lines)
        println("#partial application line(s) may have repeated!")
        for j = 1:length(maps)
            println(hsmap_convert(string(maps[j])))
            println("\n")
        end
    #============ PRE-MAIN W/ FUNCTIONS CONTAINING "WHERE" KEYWORD =========
    ========================================================================
    =#
    
    elseif occursin("where",pre_main) == true #if "where" keywords is found, do those first, then exectute the same as above
        lines = split(pre_main,"\n")
        lines = filter(x -> x != "", lines)
        wheres = findall(x -> occursin("where",x),lines)
        fun_defs = []
        fun_decs = []
        for k = 1:length(wheres)
            append!(fun_defs,[lines[wheres[k]-1]])
            append!(fun_decs,[lines[wheres[k]-2]])
        end
        names = [] #we want to store the names so we don't double count functions
        for i = 1:length(fun_decs)
            name = split(fun_decs[i],"::")[1]
            append!(names,[strip(name)])
        end
        # FINDING AND PARSING THE WHERE STATEMENTS ====
        partials = []
        for j = 1:length(fun_decs)
            fun_decs = filter(x -> !occursin("[",x),fun_decs) #removes array-like statements from function declarations
            fun_defs = filter(x -> !occursin("[",x),fun_defs) #removes array-like statements from function definition
            wh_statement = string(lines[wheres]) 
            wh_statement = split(wh_statement,"where")[j+1]
            wh_statement = filter(x -> !isspace(x),wh_statement)
            wh_statement = filter(x -> (x !="," && x !="]" && x != "''" ),wh_statement)
            
            if length(lines[wheres]) == 1  
                trunc = 2
            end
            if length(lines[wheres]) > 1
                if j == 1
                    trunc = 3
                end
                if j > 1
                    trunc = 2
                end
            end
            
            wh_statement = chop(wh_statement,head=0,tail=trunc) #trunc changes depending which index we are at. 
            wh_statement = string("where",wh_statement)
            wh_statement = split(wh_statement,",")[1]
            
            fun_blocks = string(fun_decs[j],"\n",fun_defs[j],"\n",wh_statement)
            
            where_functions(fun_blocks)
            println("")
        end
        pre_main_funcs = split(pre_main,"\n")
        pre_main_funcs = filter(x -> !occursin("where",x),pre_main_funcs) #deletes functions containing "where" in def
        lines = filter(x -> x !="", pre_main_funcs) #filters out the white spaces line by line
        custom_types = filter(x -> occursin("|",x), lines) #gets custom data types
        custom_types = filter(x -> !occursin("[",x),custom_types) #removes array statements
        funcs = filter(x -> !occursin("|",x),lines) #functions 
        funcs = filter(x -> !occursin("filter",x),funcs) #removes filter statements  
        funcs = filter(x -> !occursin("map",x),funcs) #removes map statements 
        funcs = filter(x -> !occursin("]",x),funcs)
        funcs = filter(x -> !occursin("curry",x),funcs) #removes curry statements
        funcs = filter(x -> (!occursin("(",x) || !occursin(",",x)),funcs) #removes curried function blocks
       
       
        hs_decs = filter(x -> occursin("::",x),funcs) #gets function definitons
        hs_defs = filter(x -> occursin("=",x),funcs) #gets function declarations
        
        for k = 1:length(hs_defs)
            if partial_check(string(hs_defs[k])) == false #checking to see if the partial application statements are found
                hs_defs[k] == hs_defs[k]
            end
            if partial_check(string(hs_defs[k])) == true 
                append!(partials,[hs_defs[k]]) #if partials found, append to the partials array
            end
            
        end
        if length(partials) != 0
            for i = 1:length(partials)
                hs_defs = filter(x -> x != partials[i],hs_defs) #filtering out partial application statements from hs_defs
            end
            
        end
        for j = 1:length(names)
            hs_decs = filter(x -> !occursin(names[j],x),hs_decs) #filters out declarations found during the "where" block
            hs_defs = filter(x -> !occursin(names[j],x),hs_defs) #filters out definitions found during the "where" block
        end
        
        for k = 1:length(hs_defs)
            haskell_to_julia_function(string(strip(hs_decs[k]),"\n",strip(hs_defs[k]))) #outputs functions 
            println("")
        end
       
        for i = 1:length(custom_types)
            haskell_custom_dtype(string(custom_types[i])) #outputs custom data types
            println("")
        end
        #====== LIST/ARRAY STATEMENTS =====
        ==================================#
        if occursin("[",pre_main) == true && occursin("|",pre_main) && occursin("=",pre_main) == true #find list comp arrays
            arrs = filter(x -> occursin("[",x), lines)
            arrs = filter(x -> !occursin("filter",x),arrs) #remove filter statements 
            arrs = filter(x -> !occursin("map",x),arrs) #remove map statements 
            for j = 1:length(arrs)
                println(hs_map2list(string(arrs[j]))) 
                
            end
        
        end
        println("\n")
        #==== HASKELL "FILTER" STATEMENTS ======
        ========================================#
        filters = filter(x -> occursin("filter",x),lines) 
        for k = 1:length(filters)
            println(hs_filter(string(filters[k])))
            println("\n")
        end
        #===== HASKELL "MAP" STATEMENTS =======
        =======================================#
        maps = filter(x -> occursin("map",x),lines)
        for i = 1:length(maps)
            println(hsmap_convert(string(maps[i])))
            println("\n")
        end
        
    end
    #========== PARTIAL APPLICATION BLOCK ======
    ============================================#
    for j = 1:length(partials)
        println(partial_app(string(partials[j])))
        println("\n")
    end
    
    #=============================== HASKELL MAIN BLOCK ==================================== 
    ================================================================================
    =#
    main_lines = split(main,"\n")
    main_lines = filter(x -> (!occursin("IO",x) && !occursin("do",x)),main_lines)
    for i = 1:length(main_lines)
        if occursin("putStr",main_lines[i]) == true
            hs_string_print(string(main_lines[i]))
        end
        if occursin("print",main_lines[i]) == true
            hs_print(string(main_lines[i]))
        end
    end
    println("#= for uncurried functions of the form f (x,y) there will be an extra comment in print statement =#")
end
#=
#main_parse(haskell_code)
=#
hs2jl_compiler(haskell_code)


function sumSquares(x::Int64,y::Int64)
    return x^2+y^2
end

function curriedSquares(c::Tuple)::Int64
    return sumSquares(c[1],c[2])
end


function heron(a::Float64,b::Float64,c::Float64)::Float64
   s=(a+b+c)/2
   return sqrt(s*(s-a)*(s-b)*(s-c))
end

function testFun(a::Int64,b::Int64)::Int64
   s=2*b
   return sqrt(a)+s
end

function add(x::Int64,y::Int64)::Int64
    return  x + y
end

function sphere_volume(r::Float64,h::Float64)::Float64
    return  (4/3)*(3.1415)*(r^3*h)
end

function triangleArea(b::Float64,h::Float64)::Float64
    return  (1/2)*b*h
end

function addfourInts(w::Int64,x::Int64,y::Int64,z::Int64)::Int64
    return  w+x+y+z
end

function addOne(a::Int64)::Int64
    return  a +1
end

mutable struct Suit
 Club 
 Diamond 
 Heart 
 Spade
end

mutable struct cardValue
 Two 
 Three 
 Four 
 Five 
 Six 
 Seven 
 Eight 
 Nine 
 Ten 
 Jack 
 Queen 
 King 
 Ace
end


arr = map(a -> addOne(a), collect(1:200))

arr2 = map(x -> addOne(x), collect(1:8))


arr3 = filter(x -> 

### After running the above cell, copy and past the code in the cell below and notice it is a fully functional Julia program!

In [30]:
function sumSquares(x::Int64,y::Int64)
    return x^2+y^2
end

function curriedSquares(c::Tuple)::Int64
    return sumSquares(c[1],c[2])
end


function heron(a::Float64,b::Float64,c::Float64)::Float64
   s=(a+b+c)/2
   return sqrt(s*(s-a)*(s-b)*(s-c))
end

function testFun(a::Int64,b::Int64)::Int64
   s=2*b
   return sqrt(a)+s
end

function add(x::Int64,y::Int64)::Int64
    return  x + y
end

function sphere_volume(r::Float64,h::Float64)::Float64
    return  (4/3)*(3.1415)*(r^3*h)
end

function triangleArea(b::Float64,h::Float64)::Float64
    return  (1/2)*b*h
end

function addfourInts(w::Int64,x::Int64,y::Int64,z::Int64)::Int64
    return  w+x+y+z
end

function addOne(a::Int64)::Int64
    return  a +1
end

mutable struct Suit
 Club 
 Diamond 
 Heart 
 Spade
end

mutable struct cardValue
 Two 
 Three 
 Four 
 Five 
 Six 
 Seven 
 Eight 
 Nine 
 Ten 
 Jack 
 Queen 
 King 
 Ace
end


arr = map(a -> addOne(a), collect(1:200))

arr2 = map(x -> addOne(x), collect(1:8))


arr3 = filter(x -> x%2 == 0, collect(1:500))


arr4 = filter(x -> x==1, collect(1:40))

arr5 = map(x -> addOne(x), collect(1:12))


arr6 = map(x -> addOne(x), collect(1:20))


addThree(x) = map(x -> add(3,x))


addEight(x) = map(x -> add(8,x))


println("Sum of x,y = ")
println(add(10,25))
println("Heron result = ")
println(heron(2.5,2.2,1.1))
#= for uncurried functions of the form f (x,y) there will be an extra comment in print statement =#

Sum of x,y = 
35
Heron result = 
1.208966500776594


In [31]:
sample = "add :: Float -> Float -> Float
add x y = x + y


sphereVol :: (Float,Float) -> Float
sphereVol (r,h) = (4/3)*(3.1415)*r^3*h
cur_sphere = curry sphereVol

triangleArea :: Float -> Float -> Float
triangleArea b h = (1/2)*b*h

squareArea :: Float -> Float
squareArea b = b^2

data Suit = Club | Diamond | Heart | Spade

sumIntSquares :: Integer -> Integer -> Integer
sumIntSquares x y = x^2+y^2

unit_sphere_vol = cur_sphere 1 

arr1 = [squareArea x | x <- [1..10]]
arr2 = map squareArea  [1..10]

arr3 = filter (>=4) [1..20]

main :: IO ()
main =  do
putStr \"Sum of x + y = \"
print(add 10.0 25.0)
putStr \"Volume of sphere =\"
print(sphere_volume (4.5, 4.1))"

hs2jl_compiler(sample)

function sphereVol(r::Float64,h::Float64)
    return (4/3)*(3.1415)*r^3*h
end

function cur_sphere(c::Tuple)::Float64
    return sphereVol(c[1],c[2])
end


unit_sphere_vol(x) = map(x -> cur_sphere(1,x))


function add(x::Float64,y::Float64)::Float64
    return  x + y
end

function triangleArea(b::Float64,h::Float64)::Float64
    return  (1/2)*b*h
end

function squareArea(b::Float64)::Float64
    return  b^2
end

function sumIntSquares(x::Int64,y::Int64)::Int64
    return  x^2+y^2
end

mutable struct Suit
 Club 
 Diamond 
 Heart 
 Spade
end


arr1 = map(x -> squareArea(x), collect(1:10))


arr3 = filter(x -> x>=4, collect(1:20))


#partial application line(s) may have repeated!

arr2 = map(x -> squareArea(x), collect(1:10))


unit_sphere_vol(x) = map(x -> cur_sphere(1,x))


println("Sum of x + y = ")
println(add(10.0,25.0))
println("Volume of sphere =")
println(sphere_volume(4.5,,4.1))
#= for uncurried functions of the form f (x,y) there will be an extra comment in print statement =#


In [33]:

haskell_sample = read("./haskell_sample.txt",String)

function h2j_transpile(file::String)
    f = read(file,String)
    jl_code = hs2jl_compiler(f)
    println("#= When reading from a file, if putStr statements were used in original Haskell code, there will be errors =#")
end
    

h2j_transpile("./haskell_sample.txt")

function sphereVol(r::Float64,h::Float64)
    return (4/3)*(3.1415)*r^3*h
end

function cur_sphere(c::Tuple)::Float64
    return sphereVol(c[1],c[2])
end


function add(x::Float64,y::Float64)::Float64
    return  x + y
end

function triangleArea(b::Float64,h::Float64)::Float64
    return  (1/2)*b*h
end

function squareArea(b::Float64)::Float64
    return  b^2
end

function sumIntSquares(x::Int64,y::Int64)::Int64
    return  x^2+y^2
end

mutable struct Suit
 Club 
 Diamond 
 Heart 
 Spade
end


arr1 = map(x -> squareArea(x), collect(1:10))


arr3 = filter(x -> x>=4, collect(1:20))


#partial application line(s) may have repeated!

arr2 = map(x -> squareArea(x), collect(1:10))


println("Sum of x + y = ")
))intln(add(10.0,25.0
println("Volume of sphere =")
println(cur_sphere(1.0,4.5))
#= for uncurried functions of the form f (x,y) there will be an extra comment in print statement =#
#= When reading from a file, if putStr statements were used in original Haskell code, there will be errors

In [39]:
function sphereVol(r::Float64,h::Float64)
    return (4/3)*(3.1415)*r^3*h
end

function cur_sphere(c::Tuple)::Float64
    return sphereVol(c[1],c[2])
end


function add(x::Float64,y::Float64)::Float64
    return  x + y
end

function triangleArea(b::Float64,h::Float64)::Float64
    return  (1/2)*b*h
end



function sumIntSquares(x::Int64,y::Int64)::Int64
    return  x^2+y^2
end

mutable struct Suit
 Club 
 Diamond 
 Heart 
 Spade
end


arr1 = map(x -> squareArea(x), collect(1:10))


arr3 = filter(x -> x>=4, collect(1:20))


#partial application line(s) may have repeated!

arr2 = map(x -> squareArea(x), collect(1:10))



#= for uncurried functions of the form f (x,y) there will be an extra comment in print statement =#
#= When reading from a file, if putStr statements were used in original Haskell code, there will be errors =#

MethodError: MethodError: no method matching squareArea(::Int64)
Closest candidates are:
  squareArea(!Matched::Float64) at In[38]:19